### Setting Up environment

In [ ]:
import os 
from dotenv import load_dotenv
from telegram import Update, Bot
from telegram.ext import Application , MessageHandler, filters, ContextTypes

load_dotenv()

In [ ]:
TELEGRAM_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
OLLAMA_MODEL = "qwen2.5:7b"

### Defining Tools

In [ ]:
from langchain_core.tools import tool
from duckduckgo_search import DDGS
import requests
from PyPDF2 import PdfReader
import tempfile

In [ ]:
@tool
def search_and_download_book(book_title: str) -> str:
    """
        Searches for PDF versions of books and returns the first downloadable link.
        Returns PDF path if successful, error message otherwise.
    """

    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(f"{book_title} filetype:pdf", max_results=5)]
            for result in results:
                pdf_url = result.get("href", "")
                if pdf_url.endswith(".pdf"):
                    response = requests.get(pdf_url, timeout= 10)
                    if response.status_code == 200:
                        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as f:
                            f.write(response.content)
                            return f.name
                        
        return "PDF not found"
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
@tool
def summarize_pdf(pdf_path: str) -> str:
    """
        Summarizes the content of a PDF file using the local LLM
    """
    try:
        text = ""
        with open(pdf_path, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                text += page.extract_text()[:2000]

        from langchain_ollama import ChatOllama
        llm = ChatOllama(
            model= OLLAMA_MODEL,
            temperature= 0.7,
            base_url="http://127.0.0.1:11434",
        )

        summary = llm.invoke(
            f"Summarize this book content in 5 paragraphs: {text[:3000]}"
        )
        return summary.content
    except Exception as e:
        return f"Summarization failed : {str(e)}"